In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob, os

data = pd.read_csv("../../Markers/MRK_ForceEMG_correct.csv")
trim = data[(data.trialType!="UT") & (data.trialType!="NR")].copy()
pretrim = len(trim.reset_index().index)
trim.drop(trim[((trim.rt  < trim.rt.median()) & (trim.MADrt >2))
          | ((trim.rt  > trim.rt.median()) & (trim.MADrt >7))].index, inplace=True)
trim.drop(trim[((trim.mt  < trim.mt.median()) & (trim.MADmt >2))
          | ((trim.mt  > trim.mt.median()) & (trim.MADmt >7))].index, inplace=True)
trim.drop(trim[((trim.prmt  < trim.prmt.median()) & (trim.MADprmt >2))
          | ((trim.prmt  > trim.prmt.median()) & (trim.MADprmt >7))].index, inplace=True)
trim = trim[~trim.prmt.isna()]#23 trials with na values for PMT, why ?
trim.reset_index(inplace=True)
#trim["original_contrast"] = trim["contrast"]
#trim["contrast"] = trim.apply(lambda row: 1 if row["contrast"]== 1 or row["contrast"]== 2 else 
#                              (2 if row["contrast"]== 3 or row["contrast"]== 4 else 3) , axis=1)

print((len(data.index)-pretrim)/len(data.index))
print((len(data.index)-len(trim.index))/len(data.index))
print(((len(data.index)-pretrim)/len(data.index))-(len(data.index)-len(trim.index))/len(data.index))
del data 


/home/gweindel/anaconda2/envs/future/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gweindel/anaconda2/envs/future/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


0.08231342117598042
0.2066038227175668
-0.12429040154158638


## Fitting Model

Running script DDMfiles/DDMfit.py to generate estimated parameters (extremely slow, cell after this one collects results)

In [37]:
import multiprocessing
import FDM_functions_FC as fdm

Fastdm_data = trim.copy()
Fastdm_data['FC'] = Fastdm_data.apply(lambda row: 'one' if row['FC'] == 1 else 'two', axis=1)
Fastdm_data = Fastdm_data.rename(columns = {'rt':'RT'})
Fastdm_data = Fastdm_data[(Fastdm_data.trialType != 'NR') & (Fastdm_data.trialType != 'UT')].reset_index(drop=True)
Fastdm_data.contrast = Fastdm_data.contrast.astype(str)
Fastdm_data.response = Fastdm_data.response.astype(int)
Fastdm_data = Fastdm_data[np.isfinite(Fastdm_data.prmt)]
Fastdm_data['RT'] = Fastdm_data['RT'] / 1000

Fastdm_data.drop(['index', 'Unnamed: 0', 'expdResp', 'trialType', 
       'nbrA', 'CA', 'IA', 'prmt', 'mt',
       'Apmt', 'ErringTime', 'ForceTime', 'chanPR1', 'trial', 'givenResp',
       'com', 'EMGtrial', 'MADrt', 'MADmt', 'MADprmt', 'bloc'], axis=1, inplace=True)

precision = 2
method = 'ks'#You can switch with ml, cs or ks

Free = {
    'v': ['SAT','FC',"contrast"],
    't0':['SAT','FC',"contrast"],
    'a':['SAT','FC']}

Fixed = {'zr':0.5,
         'szr':0, 
         'sv':0,
         'p':0,
         'd':0}

In [38]:
boots =  np.empty([0, 6])
nboot = 2
n = 0
id_sub = Fastdm_data.participant.unique()
for sub in id_sub:
    sub_dat = Fastdm_data[Fastdm_data.participant == sub]
    while n != nboot: 
        newsub = sub + '-' + str(n)
        newsub_dat = sub_dat.copy()
        for cell, cell_dat in sub_dat.groupby(['SAT','FC','contrast']):
            x_boots = cell_dat.copy().values
            len_row = len(x_boots)
            x_boots[:,0] = np.repeat(newsub, len_row)
            x_boots = x_boots[np.random.randint(x_boots.shape[0], size=len_row), :] #bootstraping RT/precision cells
            boots = np.concatenate((boots, x_boots), axis=0)
        n += 1     
boots = pd.DataFrame(boots, columns=["participant", "SAT", "FC", "contrast", "response", "RT"])

In [40]:
fdm.generate_files(boots, free = Free, fixed = Fixed, method=method, precision=precision)
fdm.fit(nproc=multiprocessing.cpu_count()*2)
pars = fdm.get_parameters()
pars.to_csv('Bootstrap/fit_results_%s.csv'%n)
for f in glob.glob("*.ctl"):
    os.remove(f)
for f in glob.glob("*.lst"):
    os.remove(f)
for f in glob.glob("data_*.csv"):
    os.remove(f)


Proceeding experiment_S10-0.ctl
Proceeding experiment_S10-1.ctl


In [6]:
%%time
fdm.generate_files(Fastdm_data, free = Free, fixed = Fixed, method=method, precision=precision)
fdm.fit(nproc=1)
pars = fdm.get_parameters()
pars.to_csv('fit_test.csv')

Proceeding experiment_S1.ctl
CPU times: user 92 ms, sys: 20 ms, total: 112 ms
Wall time: 5.7 s


Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/connection.py", line 379, in _re

In [10]:
%%time
fdm.generate_files(Fastdm_data, free = Free, fixed = Fixed, method=method, precision=precision)
fdm.fit(nproc=1)
pars = fdm.get_parameters()
pars.to_csv('fit_test.csv')

Proceeding experiment_S1.ctl
CPU times: user 192 ms, sys: 68 ms, total: 260 ms
Wall time: 1min 18s


Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/gweindel/anaconda2/envs/future/lib/python3.5/multiprocessing/connection.py", line 379, in _re